In [ ]:
SELECT * FROM 'meals.csv';

# Goal of the project
Analyze the food delivery company's data to uncover facts about it and improve its operations. During the project different KPI were calculated to measure company's performance.

# Project overview
The project consist of the following parts:
1) Finance metrics calculation (Revenue, Cost, and Profit).
2) User-centric metrics calculation (namely registrations, active users, growth, and retention).
3) Unit economics and distributions
4) Generating Executive report

# Part 1. Finance metrics calculation (Revenue, Cost, Profit)

## Revenue calculation

**Calculating total revenue**

The total revenue that the company generated from users was calculated using a formula for revenue and joining two tables.

In [3]:
-- Calculate revenue
SELECT SUM(meal_price*order_quantity) AS revenue
  FROM meals.csv
  JOIN orders.csv ON meals.meal_id = orders.meal_id

,revenue
0,382205.75


**Revenue per week**

In this example, calculating weekly revenue was used to evaluate the success of the marketing campaign, with the number of ads increasing each week through the end of the month. Understanding revenue per week enabled an assessment of whether there were trends in revenue growth during a specific month of the marketing campaign.

In [3]:
SELECT DATE_TRUNC('week', order_date) :: DATE AS delivr_week,
       -- Calculate revenue
       SUM(meal_price * order_quantity) AS revenue
  FROM meals.csv
  JOIN orders.csv ON meals.meal_id = orders.meal_id
-- Keep only the records in June 2018
WHERE DATE_TRUNC('month', order_date) = '2018-06-01'
GROUP BY delivr_week
ORDER BY delivr_week ASC;

,delivr_week,revenue
0,2018-05-28 00:00:00+00:00,680.75
1,2018-06-04 00:00:00+00:00,2037.75
2,2018-06-11 00:00:00+00:00,2275.00
3,2018-06-18 00:00:00+00:00,2234.50
4,2018-06-25 00:00:00+00:00,1994.50


## Cost calculation

**Calculating the Company's Top 5 Products by Overall Cost**

In this example, the goal was to determine the 5 products on which the company spent the most for stocking.

For analysis and cost calculation, the tables 'stock' and 'meals' were used. In the 'stock' table, each row represents a bulk purchase the company made to stock the meals it offers, with the stocking date, the ID of the stocked meal, and the quantity of that meal stocked. The 'meals' table contains:
- meal_price: the price the user pays the company for the meal
- meal_cost: the cost the company pays its eateries to stock the meal.

In [4]:
SELECT
  -- Calculate cost per meal ID
 meals.meal_id,
 SUM(meal_cost*stocked_quantity) AS cost
FROM meals.csv
JOIN stock.csv ON meals.meal_id = stock.meal_id
GROUP BY meals.meal_id
ORDER BY cost DESC
-- Only the top 5 meal IDs by purchase cost
LIMIT 5;

,meal_id,cost
0,4,6045.00
1,5,4212.00
2,0,3274.00
3,2,3025.75
4,6,2843.50


**Calculate average cost per month**

First, we can write a query to calculate the cost per month:

In [5]:
SELECT
  -- Calculate cost
  DATE_TRUNC('month', stocking_date)::DATE AS delivr_month,
  SUM(meal_cost*stocked_quantity) AS cost
FROM meals.csv
JOIN stock.csv ON meals.meal_id = stock.meal_id
GROUP BY delivr_month
ORDER BY delivr_month ASC;

,delivr_month,cost
0,2018-06-01 00:00:00+00:00,2222.75
1,2018-07-01 00:00:00+00:00,3594.25
2,2018-08-01 00:00:00+00:00,5365.75
3,2018-09-01 00:00:00+00:00,8480.00
4,2018-10-01 00:00:00+00:00,12488.75
5,2018-11-01 00:00:00+00:00,6277.75


Then we can wrap the previous query in a CTE:

In [ ]:
-- Declare a CTE named monthly_cost
WITH monthly_cost AS (
  SELECT
    DATE_TRUNC('month', stocking_date)::DATE AS delivr_month,
    SUM(meal_cost * stocked_quantity) AS cost
  FROM meals
  JOIN stock ON meals.meal_id = stock.meal_id
  GROUP BY delivr_month)

SELECT *
FROM monthly_cost;

Finally, we can find the average cost:

In [1]:
-- Declare a CTE named monthly_cost
WITH monthly_cost AS (
  SELECT
    DATE_TRUNC('month', stocking_date)::DATE AS delivr_month,
    SUM(meal_cost * stocked_quantity) AS cost
  FROM meals.csv
  JOIN stock.csv ON meals.meal_id = stock.meal_id
  GROUP BY delivr_month)

SELECT
  -- Calculate the average monthly cost before September
  AVG(cost)
FROM monthly_cost;

,"avg(""cost"")"
0,6404.875


## Profit calculation

The key profit metrics were determined:
- Profit per user / business unit: helps to identify the "best" users, those who bring the company the most money.
- Profit per meal: identifies the most profitable product.
- Profit per month: tracks profit over time, signaling whether or not the company is getting better at making money.

We can combine the calculations of revenue and cost using Common Table Expressions to calculate profit.

## Calculation profit per business unit

To calculate profit per business unit, we need:
- Calculate revenue per business unit in the revenue CTE.
- Calculate cost per business unit in the cost CTE.
- Join the two CTEs and calculate profit per business unit.

In [1]:
WITH revenue AS (
  -- Calculate revenue per business unit
  SELECT eatery,
         SUM(meal_price*order_quantity) AS revenue
    FROM meals.csv
    JOIN orders.csv ON meals.meal_id = orders.meal_id
   GROUP BY eatery),

  cost AS (
  -- Calculate cost per business unit
  SELECT eatery,
         SUM(meal_cost*stocked_quantity) AS cost
    FROM meals.csv
    JOIN stock.csv ON meals.meal_id = stock.meal_id
   GROUP BY eatery)

   -- Calculate profit per business unit
   SELECT revenue.eatery,
          revenue-cost AS profit
     FROM revenue
     JOIN cost ON revenue.eatery = cost.eatery
    ORDER BY profit DESC;

,eatery,profit
0,'Burgatorio',85705.00
1,'Bean Me Up Scotty',80625.75
2,'The Moon Wok',78156.50
3,'Leaning Tower of Pizza',58349.00
4,'Life of Pie',40940.25


## Calculating Profit per month

To calculate profit per month, we need:
- Calculate revenue per month in the revenue CTE.
- Calculate cost per month in the cost CTE.
- Join the two CTEs and calculate profit per month.

In [2]:
-- Set up the revenue CTE
WITH revenue AS ( 
	SELECT
		DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
		SUM(meal_price*order_quantity) AS revenue
	FROM meals.csv
	JOIN orders.csv ON meals.meal_id = orders.meal_id
	GROUP BY delivr_month),
-- Set up the cost CTE
  cost AS (
 	SELECT
		DATE_TRUNC('month', stocking_date) :: DATE AS delivr_month,
		SUM(meal_cost*stocked_quantity) AS cost
	FROM meals.csv
    JOIN stock.csv ON meals.meal_id = stock.meal_id
	GROUP BY delivr_month)
-- Calculate profit by joining the CTEs
SELECT
	revenue.delivr_month,
	revenue - cost AS profit
FROM revenue
JOIN cost ON revenue.delivr_month = cost.delivr_month
ORDER BY revenue.delivr_month ASC;

,delivr_month,profit
0,2018-06-01 00:00:00+00:00,6999.75
1,2018-07-01 00:00:00+00:00,6659.00
2,2018-08-01 00:00:00+00:00,5271.75
3,2018-09-01 00:00:00+00:00,1221.00
4,2018-10-01 00:00:00+00:00,-1968.25
5,2018-11-01 00:00:00+00:00,3645.75


# Part 2. User-Centric KPIs calculation

Calculation KPIs to track user activity such as: registrations, active users, user growth, retention rates.

## Registrations

### Registrations by month

The registrations KPI focuses on new users and counts registrations over time. It's good at measuring a company's success in attracting new users.

In [7]:
SELECT
  -- Get the earliest (minimum) order date by user ID
  user_id,
  MIN(order_date) AS reg_date
FROM updated_orders.csv
GROUP BY user_id
-- Order by user ID
ORDER BY user_id ASC;

,user_id,reg_date
0,0,2018-06-01 00:00:00+00:00
1,1,2018-06-01 00:00:00+00:00
2,2,2018-06-01 00:00:00+00:00
3,3,2018-06-01 00:00:00+00:00
4,4,2018-06-01 00:00:00+00:00
5,5,2018-06-01 00:00:00+00:00
6,6,2018-06-01 00:00:00+00:00
7,7,2018-06-01 00:00:00+00:00
8,8,2018-06-01 00:00:00+00:00
9,9,2018-06-01 00:00:00+00:00


In [5]:
-- CTE for determine reg_dates
WITH reg_dates AS (
  SELECT
    new_user_id AS user_id,
    MIN(order_date) AS reg_date
  FROM orders_modified.csv
  GROUP BY new_user_id)

SELECT
  -- Count the unique user IDs by registration month
  DATE_TRUNC('month', reg_date):: DATE AS delivr_month,
  COUNT(DISTINCT user_id) AS regs
FROM reg_dates
GROUP BY delivr_month
ORDER BY delivr_month ASC;

,delivr_month,regs
0,2018-06-01 00:00:00+00:00,123
1,2018-07-01 00:00:00+00:00,140
2,2018-08-01 00:00:00+00:00,157
3,2018-09-01 00:00:00+00:00,176
4,2018-10-01 00:00:00+00:00,199
5,2018-11-01 00:00:00+00:00,231
6,2018-12-01 00:00:00+00:00,278
7,2019-01-01 00:00:00+00:00,56


### Registrations running total

The Registrations Running Total by month is the total number of registered users in each month.

We can use this metric to know how many registered users we have overall by month.
To calculate a running total, we'll use a window function.

In [3]:
WITH reg_dates AS (
  SELECT
    new_user_id,
    MIN(order_date) AS reg_date
  FROM orders_modified_greater_variation.csv
  GROUP BY new_user_id),

  regs AS (
  SELECT
    DATE_TRUNC('month', reg_date) :: DATE AS delivr_month,
    COUNT(DISTINCT new_user_id) AS regs
  FROM reg_dates
  GROUP BY delivr_month)

SELECT
  -- Calculate the registrations running total by month
  delivr_month,
  SUM(regs) OVER (ORDER BY delivr_month ASC) AS regs_rt
FROM regs
-- Order by month in ascending order
ORDER BY delivr_month ASC;

,delivr_month,regs_rt
0,2018-06-01 00:00:00+00:00,123.0
1,2018-07-01 00:00:00+00:00,226.0
2,2018-08-01 00:00:00+00:00,337.0
3,2018-09-01 00:00:00+00:00,489.0
4,2018-10-01 00:00:00+00:00,689.0
5,2018-11-01 00:00:00+00:00,750.0
6,2018-12-01 00:00:00+00:00,775.0
7,2019-01-01 00:00:00+00:00,796.0
8,2019-02-01 00:00:00+00:00,797.0
9,2019-03-01 00:00:00+00:00,799.0


## Active users

### Monthly active users `(MAU)`

Using the active users KPI, we can focuse on existing users which counts the active users of a company's app over a time period, in our case by month. We can also use "Stickiness": the ratio of DAU (day active users) to MAU , measures how often users engage with a service on average. For example, if 's stickiness is 30%, then users use company's servise for 30% * 30 days in a month = 9 days each month on average.

To determine MAU:
- Select the month by truncating the order dates.
- Calculate MAU by counting the users for every month.
- Order by month in ascending order.

In [4]:
SELECT
  -- Truncate the order date to the nearest month
  DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
  -- Count the unique user IDs
  COUNT(DISTINCT new_user_id) AS mau
FROM orders_modified_greater_variation.csv
GROUP BY delivr_month
-- Order by month
ORDER BY delivr_month ASC;

,delivr_month,mau
0,2018-06-01 00:00:00+00:00,123
1,2018-07-01 00:00:00+00:00,226
2,2018-08-01 00:00:00+00:00,337
3,2018-09-01 00:00:00+00:00,489
4,2018-10-01 00:00:00+00:00,689
5,2018-11-01 00:00:00+00:00,750
6,2018-12-01 00:00:00+00:00,775
7,2019-01-01 00:00:00+00:00,124
8,2019-02-01 00:00:00+00:00,120
9,2019-03-01 00:00:00+00:00,119


### Tracking change MAU over time     

We will use a window function `LAG` for tracking changes in MAU over time.
Using window functions is convenient for comparing the current and previous months' monthly active users (MAUs).

The following query returns a table of MAUs and the previous month's MAU for each month:
- Set up the query by counting each month's active users and storing the results in the `MAUs` CTE.
- Then, select the month, the `mau` column, and use the `LAG` function to fetch the previous month's MAU.
- By ordering by `delivr_month` in ascending order, we can guarantee that the previous value fetched is that of the previous month.
- Since the first month has no previous month, `LAG` will return a NULL value for it; we can use `COALESCE` to set that NULL to 0.

In [4]:
WITH mau AS (
  SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT new_user_id) AS mau
  FROM orders_modified_greater_variation.csv
  GROUP BY delivr_month)

SELECT
  -- Select the month and the MAU
  delivr_month,
  mau,
  COALESCE(
    LAG(mau) OVER (ORDER BY delivr_month ASC),
  0) AS last_mau  
FROM mau
-- Order by month in ascending order
ORDER BY delivr_month ASC;

,delivr_month,mau,last_mau
0,2018-06-01 00:00:00+00:00,123,0
1,2018-07-01 00:00:00+00:00,226,123
2,2018-08-01 00:00:00+00:00,337,226
3,2018-09-01 00:00:00+00:00,489,337
4,2018-10-01 00:00:00+00:00,689,489
5,2018-11-01 00:00:00+00:00,750,689
6,2018-12-01 00:00:00+00:00,775,750
7,2019-01-01 00:00:00+00:00,124,775
8,2019-02-01 00:00:00+00:00,120,124
9,2019-03-01 00:00:00+00:00,119,120
